## Tips to tweak

- Data and preprocessing-based approaches
  - More data
  - Adjusting vocabulary size (make sure to consider the overall size of the corpus!)
  - Adjusting sequence length (more or less padding or truncation)
  - Whether to pad or truncate `pre` or `post` (usually less of an effect than the others)
- Model-based approaches
  - Adjust the number of embedding dimensions
  - Changing use of `Flatten` vs. `GlobalAveragePooling1D` (often better)
  - Considering other layers like Dropout
  - Adjusting the number of nodes in intermediate fully-connected layers

## Import

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import io
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Functions

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

## Tokenize and Encode subwords

Amazon and Yelp reviews, with their related sentiment (1 for positive, 0 for negative).\
https://www.kaggle.com/marklvl/sentiment-labelled-sentences-data-set

downloaded: reviews_sentiment.csv

In [ ]:
!wget --no-check-certificate -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

In [ ]:
csv = 'C:/Users/bruce/Desktop/GitHub/TensorFlow/download/reviews_sentiment.csv'

# extract
dataset = pd.read_csv(csv)
sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

# create a subwords dataset with tokenizer
vocab_size = 1000
max_subword_length = 5
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(sentences, vocab_size, max_subword_length=max_subword_length)

# Check that the tokenizer works appropriately
num = 5
print(sentences[num])
encoded = tokenizer.encode(sentences[num])
print(encoded)
# Separately print out each subword, decoded
for i in encoded:
  print(tokenizer.decode([i]))

# Replace sentence data with encoded subwords
# This is equivalent to `text_to_sequences` with the `Tokenizer`
for i, sentence in enumerate(sentences):
  sentences[i] = tokenizer.encode(sentence)
# Check the sentences are appropriately replaced
print(sentences[1])

## Padding and Split the data

In [ ]:
# paras
embedding_dim = 16
max_length = 50 # 100
trunc_type='post'
padding_type='post'

# Separate out the sentences and labels into training and test sets
split_ratio = 0.8
num_sentences = len(sentences)
training_size = int(num_sentences * split_ratio)
test_size = num_sentences - training_size
print("number of sentences:", num_sentences)
print("number of trains:", training_size)
print("number of tests:", test_size)

# Pad all sentences
sentences = pad_sequences(sentences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# split
training_sequences = sentences[:training_size]
testing_sequences = sentences[training_size:]
training_labels = labels[:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

## Create a Model with Embeddings

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

## Train the Model

In [ ]:
epochs = 30
history = model.fit(training_sequences, training_labels, epochs=epochs, validation_data=(testing_sequences, testing_labels))

## Visualize the training graph

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

## Visualize the network

Head to http://projector.tensorflow.org/ and load these files, then click the "Sphereize" checkbox.\
vectors (vecs.tsv)\
metadata (meta.tsv)

In [ ]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

In [ ]:
# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(0, vocab_size - 1):
  word = tokenizer.decode([word_num])
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
# Download the files
try:
  from google.colab import files
except ImportError:
  print("ImportError")
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')
  print("Downloaded")